# MobileNet处理视频文件，生成新视频

>同济大学 张子豪 2021-09-23

视频教程：同济子豪兄 https://space.bilibili.com/1900783

# 导入工具包、预训练模型

In [4]:
# 导入工具包

# 进度条库
from tqdm import tqdm

# 时间库
import time

import numpy as np

# 导入opencv-python
import cv2

# 载入图像处理、预处理、解码预测结果的辅助函数
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.mobilenet import preprocess_input, decode_predictions

# 载入预训练模型，执行图像分类

# # 导入MobileNet V1
# from tensorflow.keras.applications import MobileNet
# model = MobileNet(alpha=1.0,                      # MobileNet论文中的宽度超参数
#                 depth_multiplier=1,               # MobileNet论文中的分辨率超参数
#                 include_top=True,                 # 是否包含最后的全连接分类层
#                 weights="imagenet",               # 权重默认为ImageNet一千类图像分类
#                 classes=1000                      # 1000个类别
# )

# 导入MobileNet V2
from tensorflow.keras.applications import MobileNetV2
model = MobileNetV2(alpha=1.0,                      # MobileNet论文中的宽度超参数
                include_top=True,                 # 是否包含最后的全连接分类层
                weights="imagenet",               # 权重默认为ImageNet一千类图像分类
                classes=1000                      # 1000个类别
)

# # 导入ResNet50
# from tensorflow.keras.applications import ResNet50
# from tensorflow.keras.applications.resnet50 import preprocess_input, decode_predictions
# model = ResNet50(include_top=True,weights="imagenet",classes=1000)

# 处理单帧图像的函数

In [5]:
def process_frame(img):
    
    '''
    B站：同济子豪兄（https://space.bilibili.com/1900783）

    微信公众号：人工智能小技巧
    '''
    
    # 记录该帧开始处理的时间
    start_time = time.time()
    
    #################################################
    
    img_copy = img
    
    # BGR转RGB
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    # 缩放至224x224
    img = cv2.resize(img, (224,224))
    x = np.expand_dims(img, axis=0)
    # 预处理
    x = preprocess_input(x)

    # 获取图像分类预测结果
    preds = model.predict(x)
    results = decode_predictions(preds, top=5)
    
    # 遍历results变量中的每一个元素，写在原图左上角
    for idx, result in enumerate(results[0]): 
        text1 = '{:<10} {:>.4f}'.format(result[1], result[2])

        # !图片，添加的文字，左上角坐标，字体，字体大小，颜色，字体粗细
        img = cv2.putText(img_copy, text1, (25, 80 + 40 * idx), cv2.FONT_HERSHEY_SIMPLEX, 1.25, (255, 255, 255), 2)
    #################################################
            
    # 记录该帧处理完毕的时间
    end_time = time.time()
    # 计算每秒处理图像帧数FPS
    FPS = 1/(end_time - start_time)
    
    scaler = 1
    # 在图像上写FPS数值，参数依次为：图片，添加的文字，左上角坐标，字体，字体大小，颜色，字体粗细
    img = cv2.putText(img_copy, 'FPS  '+str(int(FPS)), (25 * scaler, 50 * scaler), cv2.FONT_HERSHEY_SIMPLEX, 1.25 * scaler, (255, 0, 255), 2 * scaler)
    
    return img_copy

# 视频逐帧处理（模板）

In [6]:
# 视频逐帧处理代码模板
# 不需修改任何代码，只需定义process_frame函数即可
# 同济子豪兄 2021-7-10

def generate_video(input_path='./videos/three-hands.mp4'):
    filehead = input_path.split('/')[-1]
    output_path = "out-" + filehead
    
    print('视频开始处理',input_path)
    
    # 获取视频总帧数
    cap = cv2.VideoCapture(input_path)
    frame_count = 0
    while(cap.isOpened()):
        success, frame = cap.read()
        frame_count += 1
        if not success:
            break
    cap.release()
    print('视频总帧数为',frame_count)
    
    # cv2.namedWindow('Crack Detection and Measurement Video Processing')
    cap = cv2.VideoCapture(input_path)
    frame_size = (cap.get(cv2.CAP_PROP_FRAME_WIDTH), cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

    # fourcc = int(cap.get(cv2.CAP_PROP_FOURCC))
    # fourcc = cv2.VideoWriter_fourcc(*'XVID')
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')
    fps = cap.get(cv2.CAP_PROP_FPS)

    out = cv2.VideoWriter(output_path, fourcc, fps, (int(frame_size[0]), int(frame_size[1])))
    
    # 进度条绑定视频总帧数
    with tqdm(total=frame_count-1) as pbar:
        try:
            while(cap.isOpened()):
                success, frame = cap.read()
                if not success:
                    break

                # 处理帧
                # frame_path = './temp_frame.png'
                # cv2.imwrite(frame_path, frame)
                try:
                    frame = process_frame(frame)
                except:
                    print('error')
                    pass
                
                if success == True:
                    # cv2.imshow('Video Processing', frame)
                    out.write(frame)

                    # 进度条更新一帧
                    pbar.update(1)

                # if cv2.waitKey(1) & 0xFF == ord('q'):
                    # break
        except:
            print('中途中断')
            pass

    cv2.destroyAllWindows()
    out.release()
    cap.release()
    print('视频已保存', output_path)

In [11]:
generate_video(input_path='./video-1.mp4')

视频开始处理 ./video-3.mp4


  1%|          | 2/300 [00:00<00:20, 14.49it/s]

视频总帧数为 301


100%|██████████| 300/300 [00:18<00:00, 16.41it/s]

视频已保存 out-video-3.mp4
